In [1]:
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objs as go

In [12]:
import os.path

data = pd.read_json('https://api.covid19api.com/all')


In [13]:
data["Date"] = pd.to_datetime(data["Date"], errors='coerce').dt.date
data["CountryProvince"] = data["Country"]+data["Province"]

In [14]:
#getting start date of contagion
min_cases = 100
temp_first = data[(data['Status']=='confirmed') & (data['Cases']>min_cases)].groupby(['CountryProvince'])['Date'].min().reset_index().rename(columns={"Date": "FirstDate"})
temp_max = data[(data['Status']=='confirmed') & (data['Cases']>=0)].groupby(['CountryProvince'])['Cases'].max().reset_index().rename(columns={"Cases": "MaxCases"})

min_d_cases = 10
temp_first_d = data[(data['Status']=='deaths') & (data['Cases']>min_d_cases)].groupby(['CountryProvince'])['Date'].min().reset_index().rename(columns={"Date": "FirstDateD"})
#temp_max = data[(data['Status']=='confirmed') & (data['Cases']>=0)].groupby(['CountryProvince'])['Cases'].max().reset_index().rename(columns={"Cases": "MaxCases"})



temp_max.head()

,CountryProvince,MaxCases
0,Afghanistan,22
1,Albania,59
2,Algeria,74
3,Andorra,39
4,Antigua and Barbuda,1


In [15]:
data = data.set_index('CountryProvince')

In [16]:
#adding start date of contagion to data
data = data.join(temp_first.set_index('CountryProvince'))
data = data.join(temp_max.set_index('CountryProvince'))
data = data.join(temp_first_d.set_index('CountryProvince'))

In [17]:
import numpy as np
data["DaysSinceConfirmed"] = np.where(data['Date']>=data['FirstDate'], (data['Date']-data['FirstDate']).dt.days, 0)
data["DaysSinceDeaths"] = np.where(data['Date']>=data['FirstDateD'], (data['Date']-data['FirstDateD']).dt.days, 0)

In [18]:
data.head()

,Country,Province,Lat,Lon,Date,Cases,Status,FirstDate,MaxCases,FirstDateD,DaysSinceConfirmed,DaysSinceDeaths
CountryProvince,,,,,,,,,,,,
Afghanistan,Afghanistan,,33.0,65.0,2020-01-22,0,recovered,NaN,22,NaN,0.0,0.0
Afghanistan,Afghanistan,,33.0,65.0,2020-01-22,0,confirmed,NaN,22,NaN,0.0,0.0
Afghanistan,Afghanistan,,33.0,65.0,2020-01-22,0,deaths,NaN,22,NaN,0.0,0.0
Afghanistan,Afghanistan,,33.0,65.0,2020-01-23,0,recovered,NaN,22,NaN,0.0,0.0
Afghanistan,Afghanistan,,33.0,65.0,2020-01-23,0,confirmed,NaN,22,NaN,0.0,0.0


In [19]:
status = "deaths"
min_total_cases = 500
temp = data[(data["MaxCases"]>=min_total_cases) ].copy()
temp.head()

,Country,Province,Lat,Lon,Date,Cases,Status,FirstDate,MaxCases,FirstDateD,DaysSinceConfirmed,DaysSinceDeaths
CountryProvince,,,,,,,,,,,,
Austria,Austria,,47.5162,14.5501,2020-01-22,0,recovered,2020-03-08,1646,NaN,0.0,0.0
Austria,Austria,,47.5162,14.5501,2020-01-22,0,confirmed,2020-03-08,1646,NaN,0.0,0.0
Austria,Austria,,47.5162,14.5501,2020-01-22,0,deaths,2020-03-08,1646,NaN,0.0,0.0
Austria,Austria,,47.5162,14.5501,2020-01-23,0,recovered,2020-03-08,1646,NaN,0.0,0.0
Austria,Austria,,47.5162,14.5501,2020-01-23,0,confirmed,2020-03-08,1646,NaN,0.0,0.0


In [20]:
provinces = temp.index.unique()
plotdata = []
for province in provinces:
    # What should go inside this Scatter call?
    province_data = temp[(temp.index == province) & (temp["Status"] == status)]
    trace = go.Scatter(x = province_data.DaysSinceDeaths, 
                       y = province_data.Cases, 
                       mode='lines', 
                       name = province,
                       text = province_data['Date'],
                       hoverinfo = "text+x+name+y"
                    )
    plotdata.append(trace)
    
layout = go.Layout(
    title = '(Logarithmic) Daily confirmed cases per Country/Province where more than {} cases have been registered'.format(min_total_cases),
    xaxis = dict(title = 'Day since first {} confirmed local case'.format(min_cases)), # x-axis label
    yaxis = dict(title = 'Confirmed cases'), # y-axis label
    yaxis_type='log'
) 

fig = go.Figure(data = plotdata, layout = layout)

fig.show()